In [10]:
from __future__ import print_function
import gtsam
import math
import numpy as np
import pickle


f = open('store.pckl', 'rb')
measurements , factors = pickle.load(f)

def Vector3(x, y, z): return np.array([x, y, z])

# 1. Create a factor graph container and add factors to it
graph = gtsam.NonlinearFactorGraph()

# 2a. Add a prior on the first pose, setting it to the origin
# A prior factor consists of a mean and a noise model (covariance matrix)
priorNoise = gtsam.noiseModel.Diagonal.Sigmas(Vector3(0.3, 0.3, 0.1))
graph.add(gtsam.PriorFactorPose2(1, gtsam.Pose2(0, 0, 0), priorNoise))

# For simplicity, we will use the same noise model for odometry and loop closures
model = gtsam.noiseModel.Diagonal.Sigmas(Vector3(1, 1, 0))

# 2b. Add odometry factors
# Create odometry (Between) factors between consecutive poses
# graph.add(gtsam.BetweenFactorPose2(1, 2, gtsam.Pose2(2, 0, 0), model))
# graph.add(gtsam.BetweenFactorPose2(2, 3, gtsam.Pose2(2, 0, math.pi / 2), model))
# graph.add(gtsam.BetweenFactorPose2(3, 4, gtsam.Pose2(2, 0, math.pi / 2), model))
# graph.add(gtsam.BetweenFactorPose2(4, 5, gtsam.Pose2(2, 0, math.pi / 2), model))

for i in factors:
        graph.add(gtsam.BetweenFactorPose2(i[0], i[1], gtsam.Pose2(i[2], i[3], i[4]), model))

# 2c. Add the loop closure constraint
# This factor encodes the fact that we have returned to the same pose. In real
# systems, these constraints may be identified in many ways, such as appearance-based
# techniques with camera images. We will use another Between Factor to enforce this constraint:
# graph.add(gtsam.BetweenFactorPose2(5, 2, gtsam.Pose2(2, 0, math.pi / 2), model))
graph.print("\nFactor Graph:\n")  # print

# 3. Create the data structure to hold the initialEstimate estimate to the
# solution. For illustrative purposes, these have been deliberately set to incorrect values
initialEstimate = gtsam.Values()
count=1
for i in measurements:
    initialEstimate.insert(count, gtsam.Pose2(i[0], i[1], i[2]))
    count = count+1
    
# initialEstimate.insert(1, gtsam.Pose2(0.5, 0.0, 0.2))
# initialEstimate.insert(2, gtsam.Pose2(2.3, 0.1, -0.2))
# initialEstimate.insert(3, gtsam.Pose2(4.1, 0.1, math.pi / 2))
# initialEstimate.insert(4, gtsam.Pose2(4.0, 2.0, math.pi))
# initialEstimate.insert(5, gtsam.Pose2(2.1, 2.1, -math.pi / 2))
initialEstimate.print("\nInitial Estimate:\n")  # print

# 4. Optimize the initial values using a Gauss-Newton nonlinear optimizer
# The optimizer accepts an optional set of configuration parameters,
# controlling things like convergence criteria, the type of linear
# system solver to use, and the amount of information displayed during
# optimization. We will set a few parameters as a demonstration.
parameters = gtsam.GaussNewtonParams()

# Stop iterating once the change in error between steps is less than this value
parameters.relativeErrorTol = 1e-5
# Do not perform more than N iteration steps
parameters.maxIterations = 100
# Create the optimizer ...
optimizer = gtsam.GaussNewtonOptimizer(graph, initialEstimate, parameters)
# ... and optimize
result = optimizer.optimize()
result.print("Final Result:\n")

parameters = gtsam.ISAM2Params()
parameters.relinearize_threshold = 0.01
parameters.relinearize_skip = 1
isam = gtsam.ISAM2(parameters)
isam.update(graph, initialEstimate)
isam.update()
currentEstimate = isam.calculate_estimate()
currentEstimate.print("Isam Results:\n")


# 5. Calculate and print marginal covariances for all variables
marginals = gtsam.Marginals(graph, result)
for i in range(1,len(measurements)+1):
        print("x"+str(i)+ " covariance:\n", marginals.marginalCovariance(i))

# print("x1 covariance:\n", marginals.marginalCovariance(1))
# print("x2 covariance:\n", marginals.marginalCovariance(2))
# print("x3 covariance:\n", marginals.marginalCovariance(3))
# print("x4 covariance:\n", marginals.marginalCovariance(4))
# print("x5 covariance:\n", marginals.marginalCovariance(5))




/usr/local/lib/python2.7/dist-packages/gtsam/__init__.py:1: RuntimeWarning: to-Python converter for boost::shared_ptr<gtsam::Cal3_S2> already registered; second conversion method ignored.
  from gtsampy import *


x1 covariance:
 [[  9.00000000e-02  -2.70539128e-39  -1.61910507e-20]
 [ -2.70539128e-39   9.00000000e-02   1.67091768e-21]
 [ -1.61910507e-20   1.67091768e-21   1.00000000e-02]]
x2 covariance:
 [[  9.73818790e-01   6.77885438e-06  -1.98102549e-04]
 [  6.77885438e-06   9.73826575e-01  -3.42189155e-04]
 [ -1.98102549e-04  -3.42189155e-04   1.00000000e-02]]
x3 covariance:
 [[  1.62526413e+00   1.63219518e-05  -2.16043650e-04]
 [  1.63219518e-05   1.62531654e+00  -7.55493246e-04]
 [ -2.16043650e-04  -7.55493246e-04   1.00000000e-02]]
x4 covariance:
 [[  2.04434431e+00   3.64951625e-05  -3.24389571e-04]
 [  3.64951625e-05   2.04446036e+00  -1.12504118e-03]
 [ -3.24389571e-04  -1.12504118e-03   1.00000000e-02]]
x5 covariance:
 [[  2.23105626e+00   6.40930492e-05  -4.29101901e-04]
 [  6.40930492e-05   2.23126095e+00  -1.49365568e-03]
 [ -4.29101901e-04  -1.49365568e-03   1.00000000e-02]]
x6 covariance:
 [[  2.92249642e+00   5.09723987e-05  -3.47928732e-04]
 [  5.09723987e-05   2.92269895e+00

In [4]:
for i in range(2,len(measurements)):
    print(currentEstimate.atPose2(i).x,currentEstimate.atPoint2(i).y)

In [9]:
#read initial estimate from libviso2 for track 1

file1 = open("/home/iiith/abhishek/Map-merging/build/data/Tr_global_1.txt",'r')
lines = file1.readlines()
measurement = []
for i in range((len(lines)+1)/6):
    measurement.append(lines[i*6+1:(i*6)+6-1])
    
measurements = []
for i in measurement:
        theta = math.asin(float(i[0].split()[2]))
        x = float(i[2].split()[3])
        y = float(i[0].split()[3])
        measurements.append((x,y,theta))

NameError: name 'math' is not defined

In [ ]:
#read initial estimate from libviso2 for track 2
file1 = open("/home/iiith/abhishek/Map-merging/build/data/Tr_global_2.txt",'r')
lines = file1.readlines()
measurement = []
for i in range((len(lines)+1)/6):
    measurement.append(lines[i*6+1:(i*6)+6-1])
    
measurements2 = []
for i in measurement:
        theta = math.asin(float(i[0].split()[2]))
        x = float(i[2].split()[3])
        y = float(i[0].split()[3])
        measurements2.append((x,y,theta))

file1.close()

In [ ]:
measurements = measurements + measurements2 #add pose of both tracks in measurements list

In [6]:
#read constraints which will be our factor graphs
# file1 = open("/home/iiith/abhishek/Map-merging/build/data/traj1_traj2_with_inliers.g2o",'r')
file1 = open("/home/iiith/codes/general/unflipped-withoutdummy-13.g2o",'r')
factors = []
for i in file1:
    if len(i.split())>2:
        values1 = i.split()[1:6]
#         import pdb;pdbD.set_trace()
        prev_frame=int(values1[0])
        next_frame=int(values1[1])
        x=float(values1[2])
        y=float(values1[3])
        theta =float(values1[4])
        factors.append((prev_frame,next_frame,x,y,theta))
file1.close()

ValueError: invalid literal for int() with base 10: '-1.59116e-09'

In [61]:
#read constraints which will be our factor graphs
# file1 = open("/home/iiith/abhishek/Map-merging/build/data/traj1_traj2_with_inliers.g2o",'r')
import numpy as np
file1 = open("/home/iiith/codes/general/unflipped-withoutdummy-13.g2o",'r')
factors = []
for i in file1:
    if len(i.split())>2:
        values1 = i.split()[1:6]
#         import pdb;pdbD.set_trace()
        prev_frame=int(values1[0])
#         next_frame=int(values1[1])
        x=float(values1[1])
        y=float(values1[2])
        theta =float(values1[3])
        factors.append([x,y,theta])
file1.close()
factors=np.array(factors)

ValueError: invalid literal for int() with base 10: 'closure'

In [2]:
import scipy.io as sio
measurements = np.array(measurements)
factors = np.array(factors)
sio.savemat('measurements.mat', {'measurements':measurements})
sio.savemat('factors.mat', {'factors':factors})

In [ ]:
import pickle
f = open('store.pckl', 'wb')
pickle.dump((measurements,factors), f)
f.close()

In [ ]:
import picklesio.savemat('np_vector.mat', {'vect':vect})
f = open('store.pckl', 'rb')
measurements , factors = pickle.load(f)

In [ ]:
factors

#plot the generated output

In [ ]:
file1=open("/home/iiith/codes/general/isam_results.txt")
results=[]
X=[]
Y=[]
for i in file1:
    if(i.split()):
        print (i.split())
        x, y, theta = (float(i.split()[3][1:-1]),float(i.split()[4][0:-1]),float(i.split()[5][:-1]))
        results.append((x,y,theta))
        X.append(x)
        Y.append(y)

In [8]:
file1=open("/home/iiith/codes/general/gtsam_results1.txt")
results=[]
X=[]
Y=[]
for i in file1:
    if(i.split()):
        print (i.split())
        x, y, theta = (float(i.split()[3][1:-1]),float(i.split()[4][0:-1]),float(i.split()[5][:-1]))
        results.append((x,y,theta))
        X.append(x)
        Y.append(y)

['Value', '1:', '(N5gtsam5Pose2E)', '(7.27434865e-18,', '-7.51759009e-18,', '2.77555756e-17)']
['Value', '2:', '(N5gtsam5Pose2E)', '(-0.0342189155,', '0.0198102549,', '2.77420231e-17)']
['Value', '3:', '(N5gtsam5Pose2E)', '(-0.0679977234,', '0.0393796729,', '-0.24640397)']
['Value', '4:', '(N5gtsam5Pose2E)', '(-0.10119643,', '0.0588960529,', '-0.246355487)']
['Value', '5:', '(N5gtsam5Pose2E)', '(-0.134399992,', '0.0780256336,', '-0.246244293)']
['Value', '6:', '(N5gtsam5Pose2E)', '(-0.133605807,', '0.0694477459,', '-0.24618816)']
['Value', '7:', '(N5gtsam5Pose2E)', '(-0.133633035,', '0.0603491607,', '-0.239419867)']
['Value', '8:', '(N5gtsam5Pose2E)', '(-0.125782204,', '0.0499768884,', '-0.241220117)']
['Value', '9:', '(N5gtsam5Pose2E)', '(-0.113875059,', '0.0289232804,', '-0.240877174)']
['Value', '10:', '(N5gtsam5Pose2E)', '(-0.0664169387,', '0.0215769973,', '-0.241115757)']
['Value', '11:', '(N5gtsam5Pose2E)', '(-0.0503603733,', '0.0056068431,', '-0.240705715)']
['Value', '12:', '(N

In [11]:
X=[]
Y=[]
for i in measurements:
    X.append(i[0])
    Y.append(i[1])

In [53]:
measurements = np.array(measurements)
factors1 = factors[0:725]
factors2 =  factors[725:]
factors_o = np.concatenate(factors2+factors1)


ValueError: operands could not be broadcast together with shapes (1199,3) (725,3) 

In [60]:
import scipy.io as sio
sio.savemat('GTSAM_tracks_gunshi',{'vect':factors_o})

In [51]:
import matplotlib.pyplot as plt
# plt.plot(measurements[:,2])
plt.plot(factors_o[:,0],factors_o[:,1])
plt.plot()
plt.ylabel('plot')
plt.show()

TypeError: list indices must be integers, not tuple

In [59]:
currentEstimate.print

In [10]:
file = open("/home/iiith/codes/SE-Sync/data/IIITH.g2o")
for i in file:
    a , b = (i.split()[1], i.split()[2])
    if int(b)-int(a)!=1:
        print "Alert:"+ i

Alert:EDGE_SE2 1112 1 -0 -0 -0

Alert:EDGE_SE2 1114 14 -0 -0 -0

Alert:EDGE_SE2 1116 48 -0 -0 -0

Alert:EDGE_SE2 1117 46 -0 -0 -0

Alert:EDGE_SE2 1118 47 -0 -0 -0

Alert:EDGE_SE2 1119 1 -0 -0 -0

Alert:EDGE_SE2 1142 11 0.114202 -0.0169816 0.058472

Alert:EDGE_SE2 1143 5 -0.0796808 -0.0267734 0.053732

Alert:EDGE_SE2 1144 12 0.00747426 -0.0256483 0.041362

Alert:EDGE_SE2 1145 15 0.0502006 -0.0127919 0.059150

Alert:EDGE_SE2 1146 13 -0.0954679 -0.0309555 0.017443

Alert:EDGE_SE2 1147 19 0.0501581 -0.0119308 0.067042

Alert:EDGE_SE2 1148 21 0.0184065 0.000641665 0.044011

Alert:EDGE_SE2 1149 29 0.174848 0.0453288 0.026831

Alert:EDGE_SE2 1150 22 -0.129225 -0.0235518 0.0078242

Alert:EDGE_SE2 1151 27 0.0191412 -0.0138067 -0.002257

Alert:EDGE_SE2 1152 31 0.118368 -0.00680855 -0.011083

Alert:EDGE_SE2 1153 30 -0.00643083 -0.0130477 -0.011300

Alert:EDGE_SE2 1154 32 0.00604333 -0.00165088 -0.013700

Alert:EDGE_SE2 1156 35 0.0538182 -0.00100853 0.02086

Alert:EDGE_SE2 1157 36 0.0491179 -0.001